In [1]:
import numpy as np
import pandas as pd
np.set_printoptions(suppress=True)

# ------------------------------------------------------
# low-level functions to parse upper triangular matrices
# ------------------------------------------------------

from itertools import chain
from scipy.spatial.distance import squareform
def iter_data(fileobj):
    for line in fileobj:
        yield from line.split()

def read_triangular_array(path):
    with open(path) as fileobj:
        first = fileobj.readline().split()
        n = len(first)
        count = int(n*(n+1)/2)
        data = chain(first, iter_data(fileobj))
        return np.fromiter(data, float, count=count)

In [4]:
ss01_ut = read_triangular_array(r'sim_sub_01.txt')
ss01_full = squareform(ss01_ut)

print("1D upper triangular format:", ss01_ut.shape)
print("2D square form format:", ss01_full.shape)

# threshold values and binarize
p_val_threshold = 0.01
ss01_out = np.where(ss01_full>p_val_threshold, 1, 0)

# load index to spatial coordinate dictionary from template dump
template_name = r'masks/mask_brain4mm_idx_xyz.dump' 
template = np.loadtxt(template_name)

nE = 0;
N = ss01_out.shape[0]
for A in range(N):
    for B in range(N):
        if ( ss01_out[i,j] == 1 ):
            a_i = (int)(template[A, 1])
            a_j = (int)(template[A, 2])
            a_k = (int)(template[A, 3])
            a_x, a_y, a_z = template[A, 4:7]
            b_i = (int)(template[B, 1])
            b_j = (int)(template[B, 2])
            b_k = (int)(template[B, 3])
            b_x, b_y, b_z = template[B, 4:7]
            if (nE == 0):
                edges_ijk = np.array([[a_i, a_j, a_k, b_i, b_j, b_k, 1]])
                edges_xyz = np.array([[a_x, a_y, a_z, b_x, b_y, b_z, 1]])
                nE += 1
            else:
                edges_ijk = np.append(edges_ijk, [[a_i, a_j, a_k, b_i, b_j, b_k, 1]]) 
                edges_xyz = np.append(edges_xyz, [[a_x, a_y, a_z, b_x, b_y, b_z, 1]]) 
                nE += 1

df_ijk = pd.DataFrame(edges_ijk)
df_xyz = pd.DataFrame(edges_xyz)
df_ijk.columns = ["a_i", "a_j", "a_k", "b_i", "b_j", "b_k", "val"]
df_xyz.columns = ["a_x", "a_y", "a_z", "b_x", "b_y", "b_z", "val"]
df_ijk.to_csv("edges_to_plot_ijk.csv", index=False)
df_xyz.to_csv("edges_to_plot_xyz.csv", index=False)

1D upper triangular format: (12497500,)
2D square form format: (5000, 5000)
